In [47]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain_openai import AzureOpenAIEmbeddings
from langchain.chat_models import AzureChatOpenAI
import os
from openai import AzureOpenAI
import openai
from dotenv import load_dotenv
load_dotenv()

True

In [44]:
default_llm = AzureChatOpenAI(
    openai_api_version=os.environ.get("AZURE_OPENAI_VERSION", "2023-07-01-preview"),
    azure_deployment=os.environ.get("AZURE_OPENAI_DEPLOYMENT", "gpt4chat"),
    azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT", "https://<your-endpoint>.openai.azure.com/"),
    api_key=os.environ.get("AZURE_OPENAI_KEY")
)

In [166]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002",
    openai_api_version=os.environ.get("AZURE_OPENAI_VERSION", "2023-07-01-preview"),
    azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT", "https://<your-endpoint>.openai.azure.com/"),
    api_key=os.environ.get("AZURE_OPENAI_KEY")
)

In [62]:
# # gets the API Key from environment variable AZURE_OPENAI_API_KEY
# client = AzureOpenAI(
#     api_version=os.environ.get("AZURE_OPENAI_VERSION"),
#     api_key=os.getenv("AZURE_OPENAI_KEY"),
#     # https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/create-resource?pivots=web-portal#create-a-resource
#     azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
# )

# completion = client.chat.completions.create(
#     model=os.environ.get("AZURE_OPENAI_DEPLOYMENT"),  # e.g. gpt-35-instant
#     messages=[
#         {
#             "role": "user",
#             "content": "How do I output all files in a directory using Python?",
#         },
#     ],
# )
# print(completion.choices[0].message.content)

In [161]:
openai_client_embeddings = AzureOpenAI(
                api_key=os.getenv("AZURE_OPENAI_KEY"),
                azure_deployment=os.environ.get("AZURE_OPENAI_DEPLOYMENT_embeddings", "embed"),
                api_version=os.environ.get("AZURE_OPENAI_VERSION", "2023-07-01-preview"),
                #azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT","https://<your-endpoint>.openai.azure.com/")
)
# use this for embeddings

In [122]:
create_args = {
            "model": "text-embedding-ada-002",
             "input": "hey there",
                    }
res2 = openai_client_embeddings.embeddings.create(**create_args)

In [135]:
ex = res2.data[0]
print(type(ex))

<class 'openai.types.embedding.Embedding'>


In [63]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

# Only keep post title, headers, and content from the full HTML.
bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

In [64]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

In [73]:
len(all_splits)

66

In [167]:
from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=all_splits, embedding=embeddings)

TypeError: Embeddings.create() got an unexpected keyword argument 'azure_endpoint'

In [74]:
from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=all_splits, embedding=embeddings)

In [78]:
type(all_splits[0])

langchain_core.documents.base.Document

In [77]:
all_splits[0].metadata

{'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/',
 'start_index': 8}

In [99]:
metadata = [split.metadata for split in all_splits]

In [84]:
response = openai_client_embeddings.embeddings.create(
    input=[split.page_content for split in all_splits],
    model="text-embedding-ada-002",
)


In [89]:
input_filex = [split.page_content for split in all_splits]
create_args = {
            "model": "text-embedding-ada-002",
             "input": input_filex
                    }
res = openai_client_embeddings.embeddings.create(**create_args)

In [95]:
type(res)

openai.types.create_embedding_response.CreateEmbeddingResponse

In [117]:
len(input_filex)

66

TypeError: FAISS.__init__() missing 1 required positional argument: 'embedding_function'

In [ ]:
# vectorstore1.add_embeddings(res.data[0], metadata)

In [141]:
# import chromadb
# CHROMA_DATA_PATH = "chroma_data/"
# EMBED_MODEL = "text-embedding-ada-002"
# COLLECTION_NAME = "demo_docs"

In [147]:
# embedding_func = embedding_functions.SentenceTransformerEmbeddingFunction(
# ...     model_name=EMBED_MODEL
# ... )

# >>> collection = client.create_collection(
# ...     name=COLLECTION_NAME,
# ...     embedding_function=embedding_func,
# ...     metadata={"hnsw:space": "cosine"},
# ... )


# collection.add(
# ...     documents=documents,
# ...     ids=[f"id{i}" for i in range(len(documents))],
# ...     metadatas=[{"genre": g} for g in genres]
# ... )

Note: you may need to restart the kernel to use updated packages.


In [154]:
def azure_openai_embed_function(text):
    # Call the Azure OpenAI service to create embeddings
    embedding_response = openai_client_embeddings.embeddings.create(
        model="text-embedding-ada-002",
        input=[text]
    )
    # Extract and return the embeddings from the response
    embeddings = embedding_response.data[0]
    return embeddings

In [156]:
vectorstore1 = FAISS.from_documents(documents=all_splits, embedding=azure_openai_embed_function)

AttributeError: 'function' object has no attribute 'embed_documents'